In [1]:
import sys
import os
from pathlib import Path

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from compute_ET_blue import (
    compute_et_blue,
    compute_volumetric_et_blue,
    postprocess_et_blue,
)

from utils import ee_utils, date_utils
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset

from typing import List, Tuple, Union

import ee
import geemap

In [4]:
ee.Initialize(project="thurgau-irrigation")

### Compute and export ET Blue raw (no postprocessing)

In [5]:
TIME_STEPS = 12
TIME_STEP_TYPE = "monthly"
YEARS = range(2022, 2023)

In [6]:
PATH_TO_AOI = "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"

aoi_feature_collection = ee.FeatureCollection(PATH_TO_AOI)
aoi_geometry = aoi_feature_collection.geometry().simplify(500)

AOI = aoi_geometry.buffer(100)

In [7]:
ET_GREEN_COLLECTION = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Zuerich/ET_green_landsat_30m_jurisdiction_monthly_ZH_2022"
).map(lambda img: back_to_float(img, 100))

date_utils.print_collection_dates(ET_GREEN_COLLECTION)
ee_utils.print_value_ranges(ET_GREEN_COLLECTION, "ET_green")

Dates of images in the collection:
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
Image 1: Min = 0.12, Max = 6.36
Image 2: Min = 5.25, Max = 17.26
Image 3: Min = 19.77, Max = 41.51
Image 4: Min = 38.16, Max = 71.45
Image 5: Min = 56.09, Max = 99.88
Image 6: Min = 67.75, Max = 116.75
Image 7: Min = 70.63, Max = 118.48
Image 8: Min = 63.42, Max = 103.79
Image 9: Min = 46.55, Max = 77.76
Image 10: Min = 25.74, Max = 45.91
Image 11: Min = 8.36, Max = 21.10
Image 12: Min = 0.15, Max = 8.22


In [10]:
# path_to_WaPOR_ET_asset = "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal_test_refactor"


# WaPOR_ET_collections = ee_utils.merge_collections(
#     years=YEARS, asset_name=path_to_WaPOR_ET_asset
# )

# ET_COLLECTION = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Zuerich/ET_WaPOR_10m_dekadal_2022"
# ).map(lambda img: back_to_float(img, 100))

ET_COLLECTION = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Zuerich/Landsat_ET_gap_filled_monthly_30m_ZH_2018-2022"
).map(lambda img: back_to_float(img, 100))

def set_negative_to_zero(img):
    return img.max(0).copyProperties(img, img.propertyNames())


ET_COLLECTION = ET_COLLECTION.map(set_negative_to_zero)

date_utils.print_collection_dates(ET_COLLECTION)
ee_utils.print_value_ranges(ET_COLLECTION, "fitted_ET")



Dates of images in the collection:
2018-01-01
2018-02-01
2018-03-01
2018-04-01
2018-05-01
2018-06-01
2018-07-01
2018-08-01
2018-09-01
2018-10-01
2018-11-01
2018-12-01
2019-01-01
2019-02-01
2019-03-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01
2019-10-01
2019-11-01
2019-12-01
2020-01-01
2020-02-01
2020-03-01
2020-04-01
2020-05-01
2020-06-01
2020-07-01
2020-08-01
2020-09-01
2020-10-01
2020-11-01
2020-12-01
2021-01-01
2021-02-01
2021-03-01
2021-04-01
2021-05-01
2021-06-01
2021-07-01
2021-08-01
2021-09-01
2021-10-01
2021-11-01
2021-12-01
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
Image 1: Min = 0.00, Max = 23.35
Image 2: Min = 0.00, Max = 32.00
Image 3: Min = 0.00, Max = 59.53
Image 4: Min = 0.00, Max = 91.67
Image 5: Min = 0.00, Max = 121.08
Image 6: Min = 0.00, Max = 138.73
Image 7: Min = 0.00, Max = 141.40
Image 8: Min = 0.00, Max = 127.94
Image 9: Min = 0.00, Max = 102.29
Ima

In [11]:
def process_and_export_et_blue_raw(
    wapor_et_list: ee.List,
    et_green_list: ee.List,
    year: int,
    aoi: ee.Geometry,
    time_steps: int = 12,
    time_step_type: str = "monthly",
) -> List[ee.batch.Task]:
    """
    Process and export ET blue raw images for a given year.

    Args:
        wapor_et_list (ee.List): List of WaPOR ET images for the year
        et_green_list (ee.List): List of ET green images for the year
        year (int): Year to process
        aoi (ee.Geometry): Area of interest to process
        time_steps (int): Number of time steps (default 12 for monthly)
        time_step_type (str): Type of time step ("monthly" or "dekadal")

    Returns:
        List[ee.batch.Task]: List of export tasks

    Raises:
        ValueError: If time_steps is not 12 or 36, or if time_step_type is invalid
    """
    if time_steps not in [12, 36]:
        raise ValueError("time_steps must be either 12 or 36")

    if time_step_type not in ["monthly", "dekadal"]:
        raise ValueError("time_step_type must be either 'monthly' or 'dekadal'")

    tasks = []

    for i in range(time_steps):
        # Get the ET images for current time step
        et_image = ee.Image(wapor_et_list.get(i))
        et_green = ee.Image(et_green_list.get(i))

        # Compute ET blue
        et_blue = compute_et_blue(et_image, et_green)
        et_blue = back_to_int(et_blue, 100)

        # Generate time step name based on type
        time_step_name = _get_time_step_name(i, time_step_type)

        # Generate export task
        task = _create_export_task(
            et_blue=et_blue,
            year=year,
            time_step_name=time_step_name,
            time_step_type=time_step_type,
            aoi=aoi,
        )

        tasks.append(task)

    return tasks


def _get_time_step_name(index: int, time_step_type: str) -> str:
    """
    Generate the time step name based on index and type.

    Args:
        index (int): Current time step index
        time_step_type (str): Type of time step ("monthly" or "dekadal")

    Returns:
        str: Formatted time step name
    """
    if time_step_type == "dekadal":
        month = index // 3 + 1
        dekad = index % 3 + 1
        return f"{month:02d}_D{dekad}"
    else:  # monthly
        month = index + 1
        return f"{month:02d}"


def _create_export_task(
    et_blue: ee.Image,
    year: int,
    time_step_name: str,
    time_step_type: str,
    aoi: ee.Geometry,
) -> ee.batch.Task:
    """
    Create an export task for an ET blue image.

    Args:
        et_blue (ee.Image): ET blue image to export
        year (int): Processing year
        time_step_name (str): Formatted time step name
        time_step_type (str): Type of time step
        aoi (ee.Geometry): Area of interest

    Returns:
        ee.batch.Task: Export task
    """
    task_name = f"ET_blue_raw_landsat_30m_ZH_{year}-{time_step_name}"

    asset_id = f"projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_landsat_30m_{time_step_type}_ZH_2022/{task_name}"

    return export_image_to_asset(
        image=et_blue,
        asset_id=asset_id,
        task_name=task_name,
        year=year,
        aoi=aoi,
        scale=30,
    )

In [ ]:
# all_tasks = []

# for year in YEARS:

#     et_list = ET_COLLECTION.filterDate(f"{year}-01-01", f"{year}-12-31").toList(
#         TIME_STEPS
#     )

#     et_green_list = ET_GREEN_COLLECTION.filterDate(
#         f"{year}-01-01", f"{year}-12-31"
#     ).toList(TIME_STEPS)

#     # Process and export
#     tasks = process_and_export_et_blue_raw(
#         wapor_et_list=et_list,
#         et_green_list=et_green_list,
#         year=year,
#         aoi=AOI,
#         time_steps=TIME_STEPS,
#         time_step_type=TIME_STEP_TYPE,  # or "dekadal" as needed
#     )

#     print(f"Year {year} processing complete. Started {len(tasks)} tasks.")

#     all_tasks.extend(tasks)

# print(f"Total tasks: {len(all_tasks)}")

Exporting ET_blue_raw_landsat_30m_ZH_2022-01 for 2022 to projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_landsat_30m_monthly_ZH_2022/ET_blue_raw_landsat_30m_ZH_2022-01
Using projection EPSG:4326 at 30m resolution
Exporting ET_blue_raw_landsat_30m_ZH_2022-02 for 2022 to projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_landsat_30m_monthly_ZH_2022/ET_blue_raw_landsat_30m_ZH_2022-02
Using projection EPSG:4326 at 30m resolution
Exporting ET_blue_raw_landsat_30m_ZH_2022-03 for 2022 to projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_landsat_30m_monthly_ZH_2022/ET_blue_raw_landsat_30m_ZH_2022-03
Using projection EPSG:4326 at 30m resolution
Exporting ET_blue_raw_landsat_30m_ZH_2022-04 for 2022 to projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_landsat_30m_monthly_ZH_2022/ET_blue_raw_landsat_30m_ZH_2022-04
Using projection EPSG:4326 at 30m resolution
Exporting ET_blue_raw_landsat_30m_ZH_2022-05 for 2022 to projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_landsat_

### Sanity check

In [13]:
et_blue_raw = ee.ImageCollection("projects/thurgau-irrigation/assets/Zuerich/ET_blue_raw_landsat_30m_monthly_ZH_2022").map(lambda img: back_to_float(img, 100))

date_utils.print_collection_dates(et_blue_raw)
ee_utils.print_value_ranges(et_blue_raw)

Dates of images in the collection:
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
Image 1: Min = -6.36, Max = 17.50
Image 2: Min = -15.08, Max = 20.60
Image 3: Min = -29.86, Max = 28.61
Image 4: Min = -55.17, Max = 40.33
Image 5: Min = -79.59, Max = 49.72
Image 6: Min = -95.07, Max = 57.14
Image 7: Min = -98.33, Max = 58.48
Image 8: Min = -87.76, Max = 51.80
Image 9: Min = -67.09, Max = 42.68
Image 10: Min = -40.72, Max = 28.79
Image 11: Min = -18.68, Max = 17.98
Image 12: Min = -8.22, Max = 21.63


In [17]:
Map = geemap.Map()

et_blue_raw_list = et_blue_raw.toList(12)

image = ee.Image(et_blue_raw_list.get(5))

vis_params = {
    "bands" : ["ET_blue"], 
    "min" : -50,
    "max" : 50,
    "palette" : ["blue", "lightblue", "green", "yellow", "orange", "red"]
}

Map.center_object(AOI, 12)
Map.addLayer(image, vis_params, "ET blue raw")

Map

Map(center=[47.41511720684401, 8.655992127451274], controls=(WidgetControl(options=['position', 'transparent_b…